In [1]:
import os

In [2]:
%pwd

'd:\\college_work\\4th year\\Sem7th\\Project\\MLOPS\\MLOPS-Wine-Quality-Prediction\\research'

In [3]:
print(f"current dir")
curr_dir = %pwd
print(f"{curr_dir}")

print('\n')
print(f"project dir")
os.chdir("../")
%pwd

current dir
d:\college_work\4th year\Sem7th\Project\MLOPS\MLOPS-Wine-Quality-Prediction\research


project dir


'd:\\college_work\\4th year\\Sem7th\\Project\\MLOPS\\MLOPS-Wine-Quality-Prediction'

Entity Creation 

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path 
    train_data_path: Path 
    test_data_path: Path 
    model_name: str 
    alpha: float 
    l1_ratio: float 
    target_column: str 

Adding the configuration also reading the params

In [5]:
from mlproject.constants import *
from mlproject.utils.common import create_directories,read_yaml

In [14]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
        
    def get_model_trainer_config(self)->ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            alpha=params.alpha,
            l1_ratio=params.l1_ratio,
            target_column=schema.name 
        )
        return model_trainer_config

Creating component the main implementation

In [15]:
import pandas as pd
import os
from mlproject import logger
from sklearn.linear_model import ElasticNet
import joblib

In [40]:
class ModelTrainer:
    def __init__(self,config:ModelTrainerConfig):
        self.config = config
        
        
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)
        
        print(test_data.columns)
        
        train_x = train_data.drop([self.config.target_column],axis=1)
        test_x = test_data.drop([self.config.target_column],axis=1)
        train_y = train_data[[self.config.target_column]]
        print(train_y.columns)
        test_y = test_data[[self.config.target_column]]
        
        

        lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=43)
        lr.fit(train_x,train_y)
        
        # joblib.dump(lr,os.path.join(self.config.root_dir,self.config.model_name))

Creating pipeline

In [41]:
try: 
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e 

[2023-12-09 23:22:02,373: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-09 23:22:02,378: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-09 23:22:02,382: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-12-09 23:22:02,383: INFO: common: created directory at: artifacts]
[2023-12-09 23:22:02,386: INFO: common: created directory at: artifacts/model_trainer]
Index(['category', 'fixed acidity', 'volatile acidity', 'citric acid',
       'residual sugar', 'chlorides', 'free sulfur dioxide',
       'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol',
       'quality'],
      dtype='object')
Index(['quality'], dtype='object')


ValueError: could not convert string to float: 'white'